In [58]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from typing import List

# 1 The Nueral Networks

In [4]:
class Net1(nn.Module):
    '''The output dimension of the full connnection layer is 100 x 100 = 10000
    '''
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 5)
        self.conv2 = nn.Conv2d(8, 32, 5)
        self.fc    = nn.Linear(15488, 10000)  # too many labels

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc(x))
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [7]:
net = Net1()

In [17]:
print(net)

Net1(
  (conv1): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(8, 32, kernel_size=(5, 5), stride=(1, 1))
  (fc): Linear(in_features=15488, out_features=10000, bias=True)
)


In [19]:
print(net.conv1.bias)
print(net.conv1.bias.grad)

Parameter containing:
tensor([ 0.1285, -0.0877,  0.0446,  0.1646,  0.0601,  0.1542,  0.1711, -0.0726],
       requires_grad=True)
tensor([ 0.0121, -0.0033,  0.0131, -0.0049,  0.0089,  0.0118,  0.0052,  0.0008])


In [42]:
params = list(net.parameters())

In [45]:
print(net.conv1.weight)

Parameter containing:
tensor([[[[-1.5852e-01,  8.8093e-02,  9.6235e-02,  7.8223e-02,  1.3234e-01],
          [-2.4708e-02,  1.6623e-01, -1.9751e-01, -1.7086e-01, -1.6195e-01],
          [-1.3942e-01,  1.7898e-01, -1.2916e-01, -9.0624e-02,  8.3965e-02],
          [ 1.7228e-01, -1.9016e-01, -1.9948e-01,  1.7751e-01, -1.3037e-01],
          [ 1.8090e-01,  1.3046e-01, -1.6858e-01, -4.2735e-02,  2.3471e-02]]],


        [[[ 4.5685e-02, -1.0702e-01, -1.5650e-02, -6.3088e-02,  1.2501e-01],
          [ 1.9547e-01, -3.4969e-02,  1.0039e-01,  1.4681e-02, -1.6749e-01],
          [ 2.5878e-02,  6.6005e-03, -1.3365e-01,  1.1471e-01, -1.7379e-01],
          [ 4.8008e-02, -6.8053e-02, -2.9220e-02, -1.6074e-01,  1.7478e-01],
          [ 1.2434e-01,  1.4895e-01,  1.5388e-01,  1.6127e-01, -6.0982e-02]]],


        [[[ 1.5571e-01, -1.0260e-01,  5.2808e-02, -9.5466e-02, -1.4221e-01],
          [ 1.0766e-01,  7.3024e-02,  8.1635e-02,  2.7965e-02, -7.1412e-02],
          [ 1.6955e-02,  6.1249e-02,  7.5589e-

In [54]:
print(net.conv1.weight)

Parameter containing:
tensor([[[[-1.5840e-01,  8.8046e-02,  9.6144e-02,  7.8180e-02,  1.3223e-01],
          [-2.4678e-02,  1.6609e-01, -1.9732e-01, -1.7077e-01, -1.6179e-01],
          [-1.3934e-01,  1.7881e-01, -1.2907e-01, -9.0562e-02,  8.3901e-02],
          [ 1.7219e-01, -1.8992e-01, -1.9932e-01,  1.7732e-01, -1.3027e-01],
          [ 1.8075e-01,  1.3039e-01, -1.6842e-01, -4.2673e-02,  2.3456e-02]]],


        [[[ 4.5648e-02, -1.0683e-01, -1.5629e-02, -6.3007e-02,  1.2484e-01],
          [ 1.9524e-01, -3.4884e-02,  1.0023e-01,  1.4640e-02, -1.6732e-01],
          [ 2.5895e-02,  6.6083e-03, -1.3350e-01,  1.1460e-01, -1.7354e-01],
          [ 4.7931e-02, -6.8008e-02, -2.9222e-02, -1.6058e-01,  1.7459e-01],
          [ 1.2422e-01,  1.4874e-01,  1.5370e-01,  1.6106e-01, -6.0823e-02]]],


        [[[ 1.5563e-01, -1.0252e-01,  5.2802e-02, -9.5414e-02, -1.4211e-01],
          [ 1.0762e-01,  7.2966e-02,  8.1599e-02,  2.7950e-02, -7.1327e-02],
          [ 1.6961e-02,  6.1232e-02,  7.5525e-

In [49]:
criterion = nn.MSELoss()  # criterion is the loss function
optimizer = optim.SGD(net.parameters(), lr=0.01)
optimizer.zero_grad()  # zero the gradient buffers

inputt = torch.randn((1, 1, 100, 100))
output = net(inputt)
target = torch.randn((1, 10000))
loss = criterion(output, target)
loss.backward()
optimizer.step()       # do the update

# Data

In [59]:
class SensorInputDataset(Dataset):
    '''Sensor reading input dataset'''
    def __init__(self, root_dir: str, labels: List, transform=None):
        '''
        Args:
            root_dir: directory with all the images
            labels:   labels of images
            transform: optional transform to be applied on a sample
        '''
        self.root_dir = root_dir
        self.transform = transform
        self.length = len(os.listdir(self.root_dir))
        self.labels = labels